## Process and apply models to single cell profiles from other data batches

We will apply classifiers to these data to prioritize samples that we predict to have specific drug-tolerance mechanisms.

In [1]:
import sys
import joblib
import pathlib
import sqlite3
import pandas as pd
from sklearn.preprocessing import StandardScaler

from pycytominer.cyto_utils import infer_cp_features

from utils.ml_utils import model_apply
from utils.single_cell_utils import process_sites, normalize_sc

sys.path.append("../0.generate-profiles")
from scripts.profile_util import load_config

In [2]:
pd.np.random.seed(1234)

In [3]:
# Set constants
batch = "2019_11_20_Batch6"
plate = 217762

feature_filter = ["Object", "Location", "Count", "Parent"]
scaler_method = "standard"
seed = 123

In [4]:
# Load locations of single cell files
config = pathlib.Path("../0.generate-profiles/profile_config.yaml")
pipeline, single_cell_files = load_config(config, append_sql_prefix=False, local=False)

In [5]:
# Load models
model_file = pathlib.Path("models", "multiclass_cloneAE_wildtype.joblib")
top_model = joblib.load(model_file)

shuffle_model_file = pathlib.Path("models", "multiclass_cloneAE_wildtype_shuffled.joblib")
top_shuffle_model = joblib.load(shuffle_model_file)

In [6]:
# Load platemap and metadata
workspace_dir = pipeline["workspace_dir"]
batch_dir = pathlib.Path(workspace_dir, "backend", batch)
metadata_dir = pathlib.Path("../0.generate-profiles", "metadata", batch)

barcode_plate_map_file = pathlib.Path(metadata_dir, "barcode_platemap.csv")
barcode_plate_map_df = pd.read_csv(barcode_plate_map_file)

plate_map_name = (
    barcode_plate_map_df
    .query("Assay_Plate_Barcode == @plate")
    .Plate_Map_Name
    .values[0]
)

plate_map_file = pathlib.Path(metadata_dir, "platemap", f"{plate_map_name}.txt")
plate_map_df = pd.read_csv(plate_map_file, sep="\t")
plate_map_df.columns = [x if x.startswith("Metadata_") else f"Metadata_{x}" for x in plate_map_df.columns]
plate_map_df.head()

,Metadata_plate_map_name,Metadata_well_position,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment
0,217762,B02,BZ017,217762,20191120-20191115-LoDensity,DMSO
1,217762,B03,WT002,217762,20191120-20191115-LoDensity,DMSO
2,217762,B04,WT008,217762,20191120-20191115-LoDensity,DMSO
3,217762,B05,WT009,217762,20191120-20191115-LoDensity,DMSO
4,217762,B06,BZ018,217762,20191120-20191115-LoDensity,DMSO


## Load single cell data

In [7]:
plate_column = pipeline["aggregate"]["plate_column"]
well_column = pipeline["aggregate"]["well_column"]

In [8]:
# Establish connection to sqlite file
single_cell_sqlite = single_cell_files[batch]["plates"][str(plate)]
conn = sqlite3.connect(single_cell_sqlite)

In [9]:
image_cols = f"TableNumber, ImageNumber, {plate_column}, {well_column}"
image_query = f"select {image_cols} from image"
image_df = (
    pd.read_sql_query(image_query, conn)
    .merge(
        plate_map_df,
        left_on=well_column,
        right_on="Metadata_well_position"
    )
    .drop(["Metadata_well_position"], axis="columns")
)

print(image_df.shape)
image_df.head()

(1020, 9)


,TableNumber,ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment
0,234604887815705591459615739336128014404,1021,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO
1,267845975164901933784670767760839550121,1081,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO
2,250335892666698831167826310881966454418,1141,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO
3,333336974385620934678639383848156659399,1201,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO
4,295132076925607667061017299156479119626,1261,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO


In [10]:
# Assert that image number is unique
assert len(image_df.ImageNumber.unique()) == image_df.shape[0]

In [11]:
# Randomly sample three sites per well to reduce number of single cells to store
sampled_image_df = image_df.groupby("Metadata_Well").apply(pd.DataFrame.sample, n=3)

sampled_image_df.head()

TableNumber  ImageNumber  \
Metadata_Well                                                            
B02           14  205114311887944755030222967061790619230         1861   
              11   50644090320183413831796116337994577555         1681   
              0   234604887815705591459615739336128014404         1021   
B03           25   29933897314022191951297828171366011617         1502   
              18  170390960487564921992417974300189969567         1082   

                  Metadata_Plate Metadata_Well  Metadata_plate_map_name  \
Metadata_Well                                                             
B02           14          217762           B02                   217762   
              11          217762           B02                   217762   
              0           217762           B02                   217762   
B03           25          217762           B03                   217762   
              18          217762           B03                   217762   

                 Metadata_clone_number  Metadata_plate_ID  \
Metadata_Well                                               
B02           14                 BZ017             217762   
              11                 BZ017             217762   
              0                  BZ017             217762   
B03           25                 WT002             217762   
              18                 WT002             217762   

                      Metadata_plate_filename Metadata_treatment  
Metadata_Well                                                     
B02           14  20191120-20191115-LoDensity               DMSO  
              11  20191120-20191115-LoDensity               DMSO  
              0   20191120-20191115-LoDensity               DMSO  
B03           25  20191120-20191115-LoDensity               DMSO  
              18  20191120-20191115-LoDensity               DMSO

In [12]:
%%time
sc_df = process_sites(
    connection=conn,
    imagenumbers=sampled_image_df.ImageNumber.tolist(),
    image_df=image_df,
    feature_filter=feature_filter,
    seed=seed,
    scaler_method=scaler_method,
    normalize=True
)

CPU times: user 19min 56s, sys: 17min 10s, total: 37min 7s
Wall time: 1h 1min 26s


In [13]:
print(sc_df.shape)
sc_df.head()

(174250, 3424)


,Metadata_TableNumber,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Cells_AreaShape_Area,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-1.158518,...,-0.287863,-0.394742,-0.276796,-0.365758,-0.168349,-0.326116,-0.354252,-0.321549,-0.292499,-0.358751
1,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-0.876328,...,1.012545,1.169435,0.571392,2.475086,2.573381,2.310072,0.989064,0.906411,0.802325,0.886704
2,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-0.736229,...,1.174563,1.235732,1.548037,1.824685,0.990363,1.062231,1.047332,0.996260,1.176186,1.153945
3,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-0.746852,...,2.171005,2.292500,1.790906,5.722734,3.059088,3.014476,2.425006,2.139544,2.013149,2.169084
4,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-0.213678,...,-0.433211,-0.425097,-0.419480,-0.384185,-0.441757,-0.423095,-0.429997,-0.435044,-0.431608,-0.431465


In [14]:
# Load test set data and reindex to match feature list
test_file = pathlib.Path("data", "single_cell_test.tsv.gz")
test_df = pd.read_csv(test_file, sep="\t")

cp_feature_order = infer_cp_features(test_df)

print(test_df.shape)
test_df.head()

(2225, 1965)


,Metadata_TableNumber,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Metadata_treatment_time,...,Nuclei_Texture_Variance_Mito_5_02,Nuclei_Texture_Variance_Mito_5_03,Nuclei_Texture_Variance_RNA_10_00,Nuclei_Texture_Variance_RNA_10_01,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,213615935970490117812207546998927362843,265,218360,D06,218360,Clone E,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,...,0.954049,0.834088,1.084193,1.078415,1.137907,1.058255,1.053407,1.123878,1.149669,1.139936
1,337567195667998632376188922851239700308,61,218360,B02,218360,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,...,0.134245,0.043566,-0.173681,-0.112904,-0.231510,-0.269831,-0.235633,-0.269732,-0.219162,-0.227415
2,44853145860897853858828255259732821643,925,218360,D06,218360,Clone E,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,...,-0.612768,-0.598612,-0.480425,-0.481877,-0.538210,-0.517666,-0.500248,-0.498740,-0.526910,-0.518281
3,287065407067703193955139587947415917934,217,218360,E08,218360,WT parental,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,...,-0.353493,-0.352171,-0.407358,-0.445175,-0.423596,-0.355138,-0.440462,-0.439416,-0.419995,-0.413208
4,208385314166099803906004723782294996203,865,218360,D06,218360,Clone E,218360,20200626-WTpAE-Lo,0.1% DMSO,13 hr,...,-0.688430,-0.692637,-0.588585,-0.592409,-0.570305,-0.575362,-0.582800,-0.577815,-0.580289,-0.575444


In [15]:
coef_file = pathlib.Path("coefficients/single_cell_multiclass_coefficients.tsv")
coef_df = pd.read_csv(coef_file, sep="\t")

print(coef_df.shape)
coef_df.head()

(1954, 4)


,feature,WT parental,Clone A,Clone E
0,Cells_AreaShape_Center_X,-0.004546,-0.012042,-0.011397
1,Cells_AreaShape_Center_Y,-0.028817,-0.032763,0.048197
2,Cells_AreaShape_Compactness,0.142783,-0.062950,-0.035459
3,Cells_AreaShape_Eccentricity,0.007360,-0.021722,0.018757
4,Cells_AreaShape_Extent,0.009453,0.016111,-0.012926


In [16]:
# Assert the feature order and the model are equivalent
assert cp_feature_order == coef_df.feature.tolist()

In [17]:
# Reindex features in the proper order before saving
meta_features = infer_cp_features(sc_df, metadata=True)
reindex_features = meta_features + cp_feature_order
sc_reindexed_df = sc_df.reindex(reindex_features, axis="columns")

print(sc_reindexed_df.shape)
sc_reindexed_df.head()

(174250, 1963)


,Metadata_TableNumber,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,Cells_AreaShape_Center_X,...,Nuclei_Texture_Variance_Mito_5_02,Nuclei_Texture_Variance_Mito_5_03,Nuclei_Texture_Variance_RNA_10_00,Nuclei_Texture_Variance_RNA_10_01,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,1.154066,...,-0.259400,-0.283441,-0.358971,-0.331362,-0.287863,-0.394742,-0.354252,-0.321549,-0.292499,-0.358751
1,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,0.128991,...,0.745973,0.798431,0.840324,1.058627,1.012545,1.169435,0.989064,0.906411,0.802325,0.886704
2,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,-1.311033,...,0.276168,0.275866,1.074907,1.194473,1.174563,1.235732,1.047332,0.996260,1.176186,1.153945
3,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,0.196236,...,1.734725,1.932412,2.230038,2.498553,2.171005,2.292500,2.425006,2.139544,2.013149,2.169084
4,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,1.482090,...,-0.320636,-0.318849,-0.432751,-0.436355,-0.433211,-0.425097,-0.429997,-0.435044,-0.431608,-0.431465


In [18]:
# Output file
sc_output_file = pathlib.Path(f"data/single_cell_{batch}_plate_{plate}_random_cells.tsv.gz")
sc_reindexed_df.to_csv(sc_output_file, sep="\t", compression="gzip", index=False)

## Apply Models

In [19]:
y_recode = {"WT parental": 0, "Clone A": 1, "Clone E": 2}
y_recode_reverse = {y: x for x, y in y_recode.items()}

In [20]:
sc_df = sc_reindexed_df.reindex(cp_feature_order, axis="columns")
meta_df = sc_reindexed_df.reindex(meta_features, axis="columns")

In [21]:
real_scores_df = model_apply(
    model=top_model,
    x_df=sc_df.fillna(0),
    meta_df=meta_df,
    y_recode=y_recode_reverse,
    data_fit="other_batch",
    shuffled=False,
    predict_proba=True
)

output_file = pathlib.Path(f"scores/{batch}_{plate}_othersinglecells.tsv.gz")
real_scores_df.to_csv(output_file, sep="\t", compression="gzip", index=False)

print(real_scores_df.shape)
real_scores_df.head()

(174250, 14)


,WT parental,Clone A,Clone E,Metadata_TableNumber,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,data_fit,shuffled
0,0.072590,0.668296,0.259113,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,False
1,0.170156,0.824304,0.005539,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,False
2,0.359179,0.633842,0.006979,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,False
3,0.004914,0.984192,0.010893,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,False
4,0.044182,0.397638,0.558181,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,False


In [22]:
shuffled_scores_df = model_apply(
    model=top_shuffle_model,
    x_df=sc_df.fillna(0),
    meta_df=meta_df,
    y_recode=y_recode_reverse,
    data_fit="other_batch",
    shuffled=True,
    predict_proba=True
)

output_file = pathlib.Path(f"scores/{batch}_{plate}_shuffled_model_othersinglecells.tsv.gz")
shuffled_scores_df.to_csv(output_file, sep="\t", compression="gzip", index=False)

print(shuffled_scores_df.shape)
shuffled_scores_df.head()

(174250, 14)


,WT parental,Clone A,Clone E,Metadata_TableNumber,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_plate_map_name,Metadata_clone_number,Metadata_plate_ID,Metadata_plate_filename,Metadata_treatment,data_fit,shuffled
0,0.321734,0.339637,0.338629,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,True
1,0.331136,0.384068,0.284796,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,True
2,0.331031,0.376710,0.292259,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,True
3,0.330624,0.380291,0.289085,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,True
4,0.353784,0.325746,0.320470,205114311887944755030222967061790619230,1861,217762,B02,217762,BZ017,217762,20191120-20191115-LoDensity,DMSO,other_batch,True
